Conversational chat with text and image

# Markdown



In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

----------------

# Dependencies

In [3]:
!pip install -q -U google-generativeai

In [4]:
!pip install -U --quiet langchain-google-genai  langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00


# Gemini model

In [5]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

In [6]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)


In [7]:
model = genai.GenerativeModel('gemini-pro-vision')

# Image

In [8]:
import PIL.Image

img = PIL.Image.open('ron.jfif')

# LLM

----------

In [9]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser

In [10]:
llm = ChatGoogleGenerativeAI(model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY)

# Using insert to add context

In [ ]:
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Who is this?",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": img},
    ]
)
message_output = llm.invoke([message])

print(message_output.content)

In [ ]:
message

In [ ]:
# we can add text into content
# we will add the message into the 2nd last position
message.content.insert(-1,  {
            "type": "text",
            "text": f"{message_output.content}",
        })

In [ ]:
new_query = "what does he do?"

message.content.insert(-1,  {
            "type": "text",
            "text": f"{new_query}",
        })

In [ ]:
message

In [ ]:
message_output = llm.invoke([message])

print(message_output.content)

In [ ]:
new_query = "who is his rival?"

message.content.insert(1,  {
            "type": "text",
            "text": f"{new_query}",
        })
message_output = llm.invoke([message])

print(message_output.content)

In [ ]:
message

# Using append didnt work

----------------

In [ ]:
model = genai.GenerativeModel('gemini-pro-vision')

messages = [
    {'role':'user',
     'parts': ["How many champions leagues has this person won.", img]}
]
response = model.generate_content(messages)

to_markdown(response.text)

In [ ]:
messages.append({'role':'model',
                 'parts':[response.text]})

prompt="Okay, how many goals has he scored in that tournament?"
messages.append({'role':'user',
                 'parts':[prompt, img]})

response = model.generate_content(messages)

to_markdown(response.text)

# Looped with insert

In [14]:
while True:
  user_msg = input("User: ")
  message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": user_msg,
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": img},
    ]
  )
  message_output = llm.invoke([message])
  print(message_output.content)

  message.content.insert(-1,  {
            "type": "text",
            "text": f"{user_msg}",
        })
  # we can add text into content
  # we will add the message into the 2nd last position
  message.content.insert(-1,  {
            "type": "text",
            "text": f"{message_output.content}",
  })
  if user_msg == "exit":
    break

User: what is his son's name
 Cristiano Ronaldo Jr.
User: how old is he
 Cristiano Ronaldo dos Santos Aveiro is a Portuguese professional footballer who plays as a forward for Serie A club Juventus and captains the Portugal national team.
User: how old is jr
 Cristiano Ronaldo dos Santos Aveiro was born 5 February 1985, is a Portuguese professional footballer who plays as a forward for Serie A club Juventus and captains the Portugal national team.
User: goofy ahh
 cristiano ronaldo
User: did ishowspeed meet this person
 Yes, IShowSpeed has met Cristiano Ronaldo. He was very excited and nervous to meet him. He took a picture with him and posted it on his social media.
User: how many likes did that picture get on instagram
 The picture got 1,000 likes on Instagram.
User: thats it?
 Cristiano Ronaldo dos Santos Aveiro is a Portuguese professional footballer who plays as a forward for Italian club Juventus and captains the Portugal national team.
User: which club does he play for?
 Cristia